### Libraries

In [370]:
import math
import numpy as np

# import pandas 
# import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

### Constants

In [371]:
# Gaussian quadrature constants 
# Notation: (w_i, x_i)
GAUSS_LAGUERRE = [
    (0.458964, 0.222847),
    (0.417, 1.188932),
    (0.113373, 2.992736),
    (0.0103992, 5.775144),
    (0.000261017, 9.837467),
    (0.000000898548, 15.982874)
]

GAUSS_LEGENDRE = [
    (0.360762, -0.661209),
    (0.467914, -0.238619),
    (0.171324, -0.171324),
    (0.171324, 0.171324),
    (0.467914, 0.238619),
    (0.360762, 0.661209)
]

### PDF, CDF, plotting functions

In [372]:
# @title Weibull distribution function -- uses Gauss-Laguerre quadratures
def f_Weibull (t, k, mu):
    denomArea = 0
    
    for node in GAUSS_LAGUERRE:
        denomArea += node[0] * (node[1]**(1/k))

    lbda = mu / denomArea

    return (k/lbda) * ((t/lbda)**(k-1)) * (math.e ** (-(t/lbda)**k))

In [373]:
# Integral of Weibull function -- uses Gauss-Legendre quadratures
def cdf (t, k, mu):
    pdf_area = 0
    
    for node in GAUSS_LEGENDRE:
        x = t/2 + (t*node[1])/2
        dx = t/2
        pdf_area += node[0] * f_Weibull(x, k, mu) * dx

    return pdf_area

In [374]:
def evaluate (f, a, b, mu05, mu1, mu2):
    x, y05, y1, y2 = [], [], [], []
    for t in range(a, b, 1):
        x.append(t)
        y05.append(f(t, 0.5, mu05))
        y1.append(f(t, 1, mu1))
        y2.append(f(t, 2, mu2))
    return x, y05, y1, y2

In [375]:
def plot (x05, x1, x2, y05, y1, y2, xtitle, ytitle, title):
    fig = go.Figure()
    fig.add_traces([
        go.Scatter(x=x05, y=y05, mode='lines', marker = {'color' : 'blue'}, name="k = 0.5"),
        go.Scatter(x=x1, y=y1, mode='lines', marker = {'color' : 'red'}, name="k = 1"),
        go.Scatter(x=x2, y=y2, mode='lines', marker = {'color' : 'magenta'}, name="k = 2")
    ])
    fig.update_layout(
        title_text=title,
        height=1080*0.5,
        width=1920*0.5,
        xaxis_title=xtitle,
        yaxis_title=ytitle
    )
    fig.show()

### Graphs

##### PDF

In [376]:
x, y05, y1, y2 = [], [], [], []

for t in range(1, 251, 1):
    t /= 100
    x.append(t)
    y05.append(f_Weibull(t, 0.5, 2))
    y1.append(f_Weibull(t, 1, 1))
    y2.append(f_Weibull(t, 2, math.sqrt(math.pi)/2))

plot(x, x, x, y05, y1, y2, "t", "alpha", "PDF (lambda = 1)")

##### CDF

In [377]:
# @title CDF values
x, y05, y1, y2 = [], [], [], []

for t in range(1, 251, 1):
    t /= 100
    x.append(t)
    y05.append(cdf(t, 0.5, 2))
    y1.append(cdf(t, 1, 1))
    y2.append(cdf(t, 2, math.sqrt(math.pi)/2))

plot(x, x, x, y05, y1, y2, "t", "alpha", "CDF (lambda = 1)")

##### S(t)

In [378]:
# @title Survival function
plot(x, x, x, [1-i for i in y05], [1-i for i in y1], [1-i for i in y2], "t", "alpha", "S(t), lambda = 1")

##### S^-1(alpha)

In [379]:
# @title Inverse survival function
plot([1-i for i in y05], [1-i for i in y1], [1-i for i in y2], x, x, x, "alpha", "t", "Inverse survival (lambda = 1)")

### Root-finding problem

In [380]:
# Inverse survivability -- solved as root-finding problem using bisection method
def bisection (a, b, alpha, mu, k):
    f = lambda t : 1 - cdf(t, k, mu) - alpha

    tol = 1.0e-9

    fa = f(a)
    fb = f(b)

    if fa == 0.0:
        return a
    
    if fb == 0.0:
        return b
    
    print("-> cdf(" + str(a) + ") - " + str(alpha) + " = " + str(fa))
    print("-> cdf(" + str(b) + ") - " + str(alpha) + " = " + str(fb))
    
    if np.sign(fa) == np.sign(fb):
        print("---> NO ROOT AT :", alpha)
        return None
    
    n = int (math.ceil (math.log(abs(b-a)/tol) / math.log(2.0)))

    print("---> iterations =", n)

    for i in range(n):
        # print("-> CDF(" + str(a) + ") - " + str(alpha) + " = " + str(fa))
        # print("-> CDF(" + str(b) + ") - " + str(alpha) + " = " + str(fb))

        c = 0.5 * (a + b)
        fc = f(c)

        if fc == 0.0:
            return c

        if np.sign(fa) != np.sign(fc):
            b = c
            fb = fc
        
        elif np.sign(fb) != np.sign(fc):
            a = c
            fa = fc

    print("---> returning c =", 0.5 * (a+b))
    return 0.5 * (a+b)

#### Evaluation

In [381]:
# Lists for values storing
x_values, y_t_05, y_t_1, y_t_2 = [], [], [], []

# Average life expectancy
mu = 78

In [382]:
# For alpha in (0,1) with 0.01 step
for alpha in range(1, 100, 1):
    alpha /= 100

    x_values.append(alpha)

    y_t_05.append(bisection(1, 501, alpha, mu, 0.5))
    y_t_1.append(bisection(1, 501, alpha, mu, 1))
    y_t_2.append(bisection(1, 501, alpha, mu, 2))

    # y_t_05.append(bisection(0, 2.5, alpha, 2, 0.5))
    # y_t_1.append(bisection(0, 2.5, alpha, 1, 1))
    # y_t_2.append(bisection(0, 2.5, alpha, math.sqrt(math.pi)/2, 2))

-> cdf(1) - 0.01 = 0.8774270624446383
-> cdf(501) - 0.01 = 0.653982246475
---> NO ROOT AT : 0.01
-> cdf(1) - 0.01 = 0.9772613849046415
-> cdf(501) - 0.01 = 0.3844479563035669
---> NO ROOT AT : 0.01
-> cdf(1) - 0.01 = 0.9898688473384537
-> cdf(501) - 0.01 = 0.14981643200952866
---> NO ROOT AT : 0.01
-> cdf(1) - 0.02 = 0.8674270624446383
-> cdf(501) - 0.02 = 0.643982246475
---> NO ROOT AT : 0.02
-> cdf(1) - 0.02 = 0.9672613849046415
-> cdf(501) - 0.02 = 0.37444795630356686
---> NO ROOT AT : 0.02
-> cdf(1) - 0.02 = 0.9798688473384537
-> cdf(501) - 0.02 = 0.13981643200952867
---> NO ROOT AT : 0.02
-> cdf(1) - 0.03 = 0.8574270624446383
-> cdf(501) - 0.03 = 0.633982246475
---> NO ROOT AT : 0.03
-> cdf(1) - 0.03 = 0.9572613849046415
-> cdf(501) - 0.03 = 0.36444795630356686
---> NO ROOT AT : 0.03
-> cdf(1) - 0.03 = 0.9698688473384537
-> cdf(501) - 0.03 = 0.12981643200952867
---> NO ROOT AT : 0.03
-> cdf(1) - 0.04 = 0.8474270624446383
-> cdf(501) - 0.04 = 0.623982246475
---> NO ROOT AT : 0.04
-

#### Plotting

In [383]:
# @title Inverse survivability plot
plot(x_values, x_values, x_values, y_t_05, y_t_1, y_t_2, "Probability (alpha)", "Time (t)", "Inverse survival (mu = 78)")

In [384]:
x, y05, y1, y2 = [], [], [], []

# PDF mu = 78
x, y05, y1, y2 = evaluate(f_Weibull, 1, 200, 78, 78, 78)
plot(x, x, x, y05, y1, y2, "t", "alpha", "PDF (mu=78)")

# CDF mu = 78
x, y05, y1, y2 = evaluate(cdf, 1, 200, 78, 78, 78)
plot(x, x, x, y05, y1, y2, "t", "alpha", "CDF (mu=78)")

# Survival mu = 78
x, y05, y1, y2 = evaluate(cdf, 1, 51, 78, 78, 78)
plot(x, x, x, [1-i for i in y05], [1-i for i in y1], [1-i for i in y2], "t", "alpha", "S(t) (mu = 78)")

# Inverse survival mu = 78
x, y05, y1, y2 = evaluate(cdf, 1, 51, 78, 78, 78)
plot([1-i for i in y05], [1-i for i in y1], [1-i for i in y2], x, x, x, "alpha", "t", "inv surv (mu = 78)")